<h1>Dcard時事版爬蟲</h1>
<h2>載入模組</h2>
<ul>
    <li>requests：建立各種 HTTP 請求，從網頁伺服器上取得想要的資料</li>
    <li>json：由於目前普遍許多的api傳過來是json格式，而dcard也不例外，所以也需要這個模組來幫忙處理</li>
    <li>csv：讀取完網頁傳過來的json資料之後，要寫入csv檔案裡</li>
</ul>

In [9]:
import requests
import json
import csv

<h2>設定需要的資訊</h2>
<p>呼叫api所需要的網址字串串接</p>
<h2>api資訊</h2>
<ul>
<li>看板資訊(meta)	https://www.dcard.tw/_api/forums</li>
<li>文章資訊(meta)	https://www.dcard.tw/_api/forums/{看板名稱}/posts</li>
<li>文章內文	https://www.dcard.tw/_api/posts/{文章編號}</li>
<li>文章內引用連結	https://www.dcard.tw/_api/{文章編號}/links</li>
<li>文章內留言	https://www.dcard.tw/_api/posts/{文章編號}/comments</li>
</ul>

In [10]:
dcardRootUrl = 'https://www.dcard.tw/_api/'#dcard的API root URL
forumsMode = 'forums/'#看板資訊
alias = 'trending/'#時事版
postsMode = 'posts/'#文章資訊
commentsMode = 'comments'#留言資訊

<p>每一篇文章都有其對應的id，越新的文章id數值是越大的，因為我們要找的文章範圍是在2018年1月到2019年5月，有測試過個別對應的id值是多少</p>

In [11]:
firstArticleID = 228016514#2018年1月的第一篇文章
lastArticleID = 231403545#2019年5月的最後一篇文章

<p>目前所爬到的文章id</p>

In [12]:
articleID = 231285499

<ul>
    <li>ForumUrl：時事版的資訊總覽，<code>popular=false</code>為以最新建立的貼文為排序，反則是以熱門程度為排序。<code>before=</code>後面則是輸入文章id值，網頁會回傳這則文章(不包含)的前30則貼文</li>
    <li>PostsUrl：直接指定要看哪一則貼文，有內容全文以及留言的資訊</li>
</ul>

In [13]:
ForumUrl = dcardRootUrl + forumsMode + alias + 'posts?popular=false&before='
PostsUrl = dcardRootUrl + postsMode

<h2>抓取資訊</h2>
<h3>使用到的方法</h3>
<p>取得時事版的資訊總覽</p>

In [14]:
def getForums():
    res = requests.get(ForumUrl + str(articleID))
    return json.loads(res.text)

<p>取得文章內容</p>

In [15]:
def getArticleContent():
    res = requests.get(PostsUrl + str(articleID))
    return json.loads(res.text)['content']

<p>取得該文章的所有留言並存進陣列</p>

In [16]:
def getComments():
    res = requests.get(PostsUrl + str(articleID) + '/' + commentsMode)
    resDict = json.loads(res.text)
    return [(resDict[index]['content'] if 'content' in resDict[index] else '') for index in range(0, len(resDict))]

<h3>使用方法請求資源並寫入csv</h3>

In [ ]:
with open('output.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['文章建立時間', '標題', '內文', '留言'])
    while articleID > firstArticleID:
        for article in getForums():
            createTime = article['createdAt']
            title = article['title']
            articleID = article['id']
            print(str(articleID) + ' ' + str(createTime) + ':' + title)#測試用
            writer.writerow([createTime, title, getArticleContent(), getComments()])
    csvfile.close()